# Creating APIs in Python using Flask

APIs can be very useful. We can get information about a user, get statistics, and much more. But what if we want to create our own API? Here I will show you how to do that.

Let's first install Flask. For me, Flask is already installed in my Anaconda distribution.

In [1]:
pip install Flask

Note: you may need to restart the kernel to use updated packages.


Now let's import it and create a new app!

In [2]:
import flask
app = flask.Flask(__name__)

Let's make our API super advanced. You will be able to go to a website to create an API key, but each IP address can only create one key.

Let's create our API key generator. We'll store our keys to a file named `keys.txt` and make each key 25 characters long. We will also make sure we do not get any duplicate keys, even though the chance of that occurance is virtually impossible. In addition, we want to store the IP address of the user who creates the key so that if they try to create another one we give them back their previous key.

In [3]:
# We need to use "choice" to pick random characters from our character list.
from random import choice

def key_dict(txt):
    """
    Converts a file in the format "29afd352e196484b83d53d1ac071685b - 156.435.35.24" to a dictionary like this:
    {
        "156.435.35.24": "29afd352e196484b83d53d1ac071685b"
    }
    """
    # Turns text into a list containing individual lines
    txt = txt.split("\n")
    # Removes the last line because it will always be "\n"
    txt.pop()
    # Empty IP address list to append addresses to
    ip_addrs = []
    # Appends IP addresses
    for line in txt:
        ip_addrs.append(line.split(" - ")[1])
    # Empty API key list to append API keys to
    api_keys = []
    # Appends API keys
    for line in txt:
        api_keys.append(line.split(" - ")[0])
    # Create output dictionary
    output = {}
    # Create dictionary content
    for ip_addr in ip_addrs:
        # Matches up IP addresses and API keys
        output[ip_addr] = api_keys[ip_addrs.index(ip_addr)]
    # Return output dictionary
    return output
        
def create_key(ip):
    """
    Used to create API keys. Takes in an IP address so that if the user has already made an API key they are given back their old one.
    """
    try:
        # Opens "keys.txt"
        with open("keys.txt", "r") as f:
            # Sets "reference_dict" to the IP to key dictionary of the file
            reference_dict = key_dict(f.read())
            # Checks if IP address already has a key
            if ip in reference_dict.keys():
                # If so, the key is returned
                return reference_dict[ip]
    except FileNotFoundError:
        pass
    # List of characters we want in our API key
    chars = "abcdefghijklmnopqrstuvwxyz1234567890"
    # Declaration of "key" variable so that we can add characters to it without a NameError
    key = ""
    # We loop 25 times to make a 25 character API key
    for i in range(25):
        # Adds random character to API key
        key += choice(chars)
    # Makes sure the API key does not already exist
    if not(key in reference_dict.values()):
        with open("keys.txt", "a") as f:
            # Append API key to "keys.txt"
            f.write(key + " - " + ip + "\n")
        # Return API key
        return key
    else:
        # Re-runs the function
        return create_key(ip)

Great! Let's now make a website that users can go to in order to create an API key.

In [4]:
@app.route("/generate-key")
def key_page():
    # Returns the output of the "create_key" function with the visitor IP as a parameter
    return create_key(flask.request.remote_addr)

Amazing. Now it's time to work on our main API page. For this example, I will be making an API that gives the servers ping to a given webpage in milliseconds, and we will also give the web server's IP address.

Let's create a function that returns a website's ping in milliseconds. Using this method we can actually also find the web server's IP address, so we can tackle two things at once. I will return the values as JSON.

In [5]:
import subprocess as sp
import json

def api_output(url):
    """
    Returns a JSON string with the server's ping the the URL in milliseconds and the IP of the URL.
    """
    # Removes beginning protocol because the program fails with it
    url = url.replace("https://", "")
    url = url.replace("http://", "")
    # Checks the output of the "ping" command being ran on the URL
    p = sp.check_output(["ping", url]).strip()
    # Python dictionary with outputs
    output = {
        "ping": int(p.decode("UTF-8").split("\n")[-1].split(" ")[-1].replace("ms", "")),
        "server-ip": p.decode("UTF-8").split("\n")[0].split(" ")[2].strip("[").strip("]")
    }
    # Returns dictionary as JSON
    return json.dumps(output)

Let's create the web request handler for our main page!

In [6]:
@app.route("/api", methods=['GET'])
def api_page():
    try:
        with open("keys.txt", "r") as f:
            txt = f.read()
    except FileNotFoundError:
        return "No API keys exist"
    # Turns text into a list containing individual lines
    txt = txt.split("\n")
    # Removes the last line because it will always be "\n"
    txt.pop()
    # Empty API key list to append API keys to
    api_keys = []
    # Appends API keys
    for line in txt:
        api_keys.append(line.split(" - ")[0])
    # Checks for valid API key
    if flask.request.args.get("key") in api_keys:
        # Checks if no URL was provided
        if flask.request.args.get("url") == None:
            return "No URL provided"
        else:
            # Returns a JSON output with information about the URL
            return api_output(flask.request.args.get("url"))
    # Checks if no API key was provided
    elif flask.request.args.get("key") == None:
        return "No API key provided"
    else:
        return "Invalid API key"

We are now done! Time to run our web server! I will be running mine on port 8080.

In [ ]:
app.run(host="localhost", port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2021 08:23:12] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [25/Jan/2021 08:23:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Jan/2021 08:23:18] "GET /api HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2021 08:23:48] "GET /create-key HTTP/1.1" 404 -
127.0.0.1 - - [25/Jan/2021 08:23:55] "GET /generate-key HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2021 08:24:00] "GET /generate-key HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2021 08:24:00] "GET /generate-key HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2021 08:24:23] "GET /api?key=el6qw6wbtw1pf3fioa33f49gc&url=lakeoswecode.org HTTP/1.1" 200 -
